# Mask R-CNN Beginner's Tutorial


## Install Google drive


In [3]:
from google.colab import drive
import sys, datetime

# Connect Google drive with authentication
drive.mount("/content/gdrive", force_remount=True)

# Set root dir of the project
ROOT_DIR = "/content/gdrive/MyDrive/MaskRCNN_Tutorial/"
sys.path.insert(0, ROOT_DIR)
# List file information in the path
%ls $ROOT_DIR

# Set training phase with time stamp
train_phase = "OralCancer_{:%Y%m%dT%H%M}".format(datetime.datetime.now())
print("\nSet training phase: {}".format(train_phase))

Mounted at /content/gdrive
DATASET/  dataset_split.py  mrcnn/  OUTPUT/  README.md
test/  train/  val/

Set training phase: OralCancer_20220618T1839


## Import required library

In [ ]:
# Import library
import json
import skimage.draw
import numpy as np
from google.colab.patches import cv2_imshow

# Import Mask RCNN
from mrcnn import visualize
from mrcnn.config import Config
from mrcnn import model as modellib, utils

## Configurations

Set model configuration by the ModelConfig class.

In [ ]:
class ModelConfig(Config):
    """Configuration for training on the oral dataset.
    Derives from the base Config class and overrides some values.
    """
    # Give the configuration a recognizable name
    NAME = "OralCancer"

    # Set specific dataset dir
    DATASET_DIR = "DATASET/DATA_220617"
    
    # Number of training epochs
    EPOCH = 10
    
    # Number of training steps per epoch
    STEPS_PER_EPOCH = 10

    # Weight
    WEIGHT = "coco"
    # [Option] "coco", "imagenet", "last", "<trained>"
    
    # Directory to save logs and model checkpoints
    LOGS_DIR = "LOG"
    
    # Batch size
    # We use a GPU with 12GB memory, which can fit two images.
    # Adjust down if you use a smaller GPU.
    IMAGES_PER_GPU = 3

    # Number of classes (including background)
    NUM_CLASSES = 1 + 3  # Background + number of classes

    # Skip detections with < 90% confidence
    DETECTION_MIN_CONFIDENCE = 0.9


## Dataset

In [ ]:
class OralDataset(utils.Dataset):

    def load_dataset(self, dataset_dir, subset):
        """ Load a subset of the oral dataset.
        dataset_dir: Root directory of the dataset.
        subset: Subset to load: train or val
        """
        # Add classes according to the numbe of classes required to detect
        self.add_class("OralCancer", 1, "G")
        self.add_class("OralCancer", 2, "Y")
        self.add_class("OralCancer", 3, "R")
        print("\n[INFO] self.class_info: \n{}".format(self.class_info))
        
        # Train or validation dataset?
        assert subset in ["train", "val", "test"]
        dataset_dir = os.path.join(dataset_dir, subset)
        
        # Load annotations
        # VGG Image Annotator (up to version 1.6) saves each image in the form:
        # { 'filename': '28503151_5b5b7ec140_b.jpg',
        #   'regions': {
        #       '0': {
        #           'region_attributes': {},
        #           'shape_attributes': {
        #               'all_points_x': [...],
        #               'all_points_y': [...],
        #               'name': 'polygon'}},
        #       ... more regions ...
        #   },
        #   'size': 100202
        # }
        # We mostly care about the x and y coordinates of each region
        # Note: In VIA 2.0, regions was changed from a dict to a list.
        annotations = json.load(open(os.path.join(dataset_dir, subset+".json")))
        annotations = list(annotations.values())  # don't need the dict keys
        
        # The VIA tool saves images in the JSON even if they don't have any
        # annotations. Skip unannotated images.
        annotations = [a for a in annotations if a['regions']]
        
        # Add images
        for a in annotations:
            # Get the x, y coordinaets of points of the polygons that make up
            # the outline of each object instance. These are stores in the
            # shape_attributes (see json format above)
            # The if condition is needed to support VIA versions 1.x and 2.x.
            if type(a['regions']) is dict:
                polygons = [r['shape_attributes'] for r in a['regions'].values()]
            else:
                polygons = [r['shape_attributes'] for r in a['regions']]
            
            #===================================================================
            label_cls = [r['region_attributes']['OralCancer'] for r in a['regions']]
            class_ids = {'G': 1,
                    'Y': 2,
                    'R': 3}
            label_ids = [class_ids[a] for a in label_cls]
            #===================================================================
            
            # load_mask() needs the image size to convert polygons to masks.
            # Unfortunately, VIA doesn't include it in JSON, so we must read
            # the image. This is only managable since the dataset is tiny.
            image_path = os.path.join(dataset_dir, a['filename'])
            image = skimage.io.imread(image_path)
            height, width = image.shape[:2]
            
            self.add_image(
                "OralCancer",
                image_id=a['filename'],  # use file name as a unique image id
                class_ids=label_ids,
                path=image_path,
                width=width,height=height,
                polygons=polygons
            )
    
    def load_mask(self, image_id):
        """Generate instance masks for an image.
       Returns:
        masks: A bool array of shape [height, width, instance count] with
            one mask per instance.
        class_ids: a 1D array of class IDs of the instance masks.
        """
        # If not a oral dataset image, delegate to parent class.
        image_info = self.image_info[image_id]
        if image_info["source"] != "OralCancer":
            return super(self.__class__, self).load_mask(image_id)
        
        class_ids = image_info["class_ids"]
        class_ids = np.array(class_ids, dtype=np.int32)

        # Convert polygons to a bitmap mask of shape
        # [height, width, instance_count]
        info = self.image_info[image_id]
        mask = np.zeros([info["height"], info["width"], len(info["polygons"])],
                        dtype=np.uint8)
        for i, p in enumerate(info["polygons"]):
            # Get indexes of pixels inside the polygon and set them to 1
            rr, cc = skimage.draw.polygon(p['all_points_y'], p['all_points_x'])
            mask[rr, cc, i] = 1
        
        return mask.astype(np.bool_), class_ids

    def image_reference(self, image_id):
        """Return the path of the image."""
        info = self.image_info[image_id]
        if info["source"] == "OralCancer":
            return info["path"]
        else:
            super(self.__class__, self).image_reference(image_id)

In [ ]:
def train(model):
    """Train the model."""
    # Training dataset.
    dataset_train = OralDataset()
    dataset_train.load_dataset(ROOT_DIR+config.DATASET_DIR, "train")
    dataset_train.prepare()

    # Validation dataset
    dataset_val = OralDataset()
    dataset_val.load_dataset(ROOT_DIR+config.DATASET_DIR, "val")
    dataset_val.prepare()

    # *** This training schedule is an example. Update to your needs ***
    # Since we're using a very small dataset, and starting from
    # COCO trained weights, we don't need to train too long. Also,
    # no need to train all layers, just the heads should do it.
    print("Training network heads")
    model.train(dataset_train, dataset_val,
          learning_rate=config.LEARNING_RATE,
          epochs=config.EPOCH,
          layers='heads'
    )

## Train on the dataset

In [ ]:
if __name__ == '__main__':
    
    # Configurations
    config = ModelConfig()
    config.display()

    # Create model
    model = modellib.MaskRCNN(mode="training", config=config, model_dir=config.LOGS_DIR)

    # Select weights file to load
    if config.WEIGHT.lower() == "coco":
        weights_path = COCO_WEIGHTS_PATH
        # Download weights file
        if not os.path.exists(weights_path):
            utils.download_trained_weights(weights_path)
        # Exclude the last layers because they require a matching
        # number of classes
        model.load_weights(weights_path, by_name=True, exclude=[
                  "mrcnn_class_logits", "mrcnn_bbox_fc",
                  "mrcnn_bbox", "mrcnn_mask"])
    elif config.WEIGHT.lower() == "imagenet":
        # Start from ImageNet trained weights
        weights_path = model.get_imagenet_weights()
    elif config.WEIGHT.lower() == "last":
        # Find last trained weights
        weights_path = model.find_last()
    else:
        weights_path = config.WEIGHT
    
    # Load weights
    print("Loading weights ", weights_path)
    model.load_weights(weights_path, by_name=True)
    
    # Train model
    train(model)

## Exhibit training curves in tensorboard:

In [ ]:
%load_ext tensorboard
%tensorboard --logdir config.LOG_DIR